---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
'''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
def get_list_of_university_towns():

    # open the test file
    univ = open('university_towns.txt', 'r') 

    state=''
    region=''
    town_list=[]

    #read the text file as book, line by line
    with univ as book:
        # read line by line
        for count, line in enumerate(book):

             #find the position of '[edit]' in each line
            pos=line.find('[edit]')

            if (pos!=-1):
                # if [edit] is found, the line contains state name
                state=line[:pos].rstrip() #strip state name of the white spaces

            else:
                # if [edit] is not found, the line contains town name with university name
                pos1=line.find('(') # find position of '(' where name of university starts
                region=line[:pos1].rstrip() # strip town name of white spaces

            if(pos==-1):
                # if line doesn't have state name, add town name to the list with corresponding state name 
                town_list=town_list+[[state,region]]

    #convert list into a dataframe
    df=pd.DataFrame(town_list,columns=['State','RegionName'])
    df.loc[96,'RegionName']='Champaign–Urbana'
    
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
def get_recession_start():

    GDP=pd.read_excel('gdplev.xls')

    # read the excel file from line 7 onwards
    GDP=GDP[7:]

    # rename columns for legibility
    GDP=GDP.rename(columns={'Current-Dollar and "Real" Gross Domestic Product' : 'Year','Unnamed: 1': 'GDP_c','Unnamed: 2': 'GDP_09','Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP_cq','Unnamed: 6': 'GDP_09q'}).reset_index()

    # drop some columns which are not needed 
    # subset quarterly data for relevant years 
    GDP=GDP.drop(['index','Unnamed: 3'], axis=1).iloc[212:,3:6]

    count=1 # count number of quarters of declining GDP
    prev1=0 # temporary variables to exchange data
    prev2=0 # temporary variables to exchange data
    rec_start=0

    # loop over each quarter
    for each in GDP['GDP_09q']:
        if (count>2):
            # if this isnt the first two quarter of the relevant period
            if ((each-prev1<0) & (prev1-prev2<0)):
                # if GDP has been declining for past two quarters, store the quarter value
                rec_start=GDP.loc[GDP['GDP_09q']==each].index[0]-1
                break
            else:
                # if GDP has not been declining for past two quarters, then exchnage values for comparison next quarter
                prev2=prev1
                prev1=each
        else:
            if(count==1):
                # if this is first quarter of the relevant period, store this quarter value for comparison next quarter
                prev2=each

            if(count==2):
                # if this is second quarter of the relevant period, store this quarter value for comparison next quarter
                prev1=each

        count+=1   

    return GDP.loc[rec_start]['Quarter']
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():

    GDP=pd.read_excel('gdplev.xls')

    # read the excel file from line 7 onwards
    GDP=GDP[7:]

    # rename columns for legibility
    GDP=GDP.rename(columns={'Current-Dollar and "Real" Gross Domestic Product' : 'Year','Unnamed: 1': 'GDP_c','Unnamed: 2': 'GDP_09','Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP_cq','Unnamed: 6': 'GDP_09q'}).reset_index()

    # drop some columns which are not needed 
    # subset quarterly data for relevant years 
    GDP=GDP.drop(['index','Unnamed: 3'], axis=1).iloc[212:,3:6]

    count=1 # count number of quarters of declining GDP
    prev1=0 # temporary variables to exchange data
    prev2=0 # temporary variables to exchange data
    rec_start=0

    # loop over each quarter
    for each in GDP['GDP_09q']:
        if (count>2):
            # if this isnt the first two quarter of the relevant period
            if ((each-prev1<0) & (prev1-prev2<0)):
                # if GDP has been declining for past two quarters, store the quarter value
                rec_start=GDP.loc[GDP['GDP_09q']==each].index[0]-1
                # end the loop
                break
            else:
                # if GDP has not been declining for past two quarters, then exchnage values for comparison next quarter
                prev2=prev1
                prev1=each
        else:
            if(count==1):
                # if this is first quarter of the relevant period, store this quarter value for comparison next quarter
                prev2=each

            if(count==2):
                # if this is second quarter of the relevant period, store this quarter value for comparison next quarter
                prev1=each

        count+=1   
    
    rec_end=0
    count=1 # count number of quarters of declining GDP
    prev1=0 # temporary variables to exchange data
    prev2=0 # temporary variables to exchange data

    for each in GDP['GDP_09q']:
        if (count>2):
            # if this isnt the first two quarter of the relevant period
            if ((each-prev1>0) & (prev1-prev2>0) & ((GDP.loc[GDP['GDP_09q']==each].index[0]-2)>rec_start)):
                # if GDP has been increasing for past two quarters,
                # and the quarter is ahead of the quarter where recession began, store the quarter value
                rec_end=GDP.loc[GDP['GDP_09q']==each].index[0]
                # end the loop
                break
            else:
                # if GDP has not been increasing for past two quarters, then exchnage values for comparison next quarter
                prev2=prev1
                prev1=each

        else:
            if(count==1):
                # if this is first quarter of the relevant period, store this quarter value for comparison next quarter
                prev2=each
            if(count==2):
                # if this is second quarter of the relevant period, store this quarter value for comparison next quarter
                prev1=each

        count+=1   
    return GDP.loc[rec_end]['Quarter']
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():

    GDP=pd.read_excel('gdplev.xls')

    # read the excel file from line 7 onwards
    GDP=GDP[7:]

    # rename columns for legibility
    GDP=GDP.rename(columns={'Current-Dollar and "Real" Gross Domestic Product' : 'Year','Unnamed: 1': 'GDP_c','Unnamed: 2': 'GDP_09','Unnamed: 4':'Quarter', 'Unnamed: 5':'GDP_cq','Unnamed: 6': 'GDP_09q'}).reset_index()

    # drop some columns which are not needed 
    # subset quarterly data for relevant years 
    GDP=GDP.drop(['index','Unnamed: 3'], axis=1).iloc[212:,3:6]

    # get index of recession start quarter
    start=GDP.loc[get_recession_start()==GDP['Quarter']].index[0]

    # get index of recession end quarter
    end=GDP.loc[GDP['Quarter']==get_recession_end()].index[0]

    # get index of quarter in between recession start and end where GDP was minimum
    i=GDP.loc[GDP['GDP_09q']==np.min(GDP.loc[start:end]['GDP_09q'])].index[0]

    # retun the quarter for the recession bottom quarter
    return GDP.loc[i]['Quarter']
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():

    Housing=pd.read_csv('City_Zhvi_AllHomes.csv')
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    # map the state codes in Housing to the State names in above dictionary
    Housing['STATE']=Housing['State'].map(states)

    # set the combination of state and region name as index
    Housing=Housing.set_index(['STATE','RegionName'])

    # drop irrelevant data from DataFrame
    Housing.drop(Housing.columns[[item for item in range(0,50)]], axis=1, inplace=True)
    
    # rename the columns to date time format
    Housing= Housing.rename(columns=pd.to_datetime)

    # modify the dataframe to get quarterly mean of prices
    Housing = Housing.resample('Q',axis=1).mean()

    # rename the columns to quarter format
    Housing = Housing.rename(columns=lambda x: str(x.to_period('Q')).lower())
    return Housing

    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
STATE,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [8]:
def run_ttest():
   # get the quarter value before recession start
   start=get_recession_start()[0:5]+(str(int(get_recession_start()[5:6])-1))

   # get quarter value for recession bottom
   bottom=get_recession_bottom()

   # get the housing data from above
   Housing=convert_housing_data_to_quarters()

   # get university town list and set index to state and region name
   univ=get_list_of_university_towns().set_index(['State', 'RegionName'])

   # calculate ratio of prices before start and at the bottom
   Housing['ratio']=Housing[start].div(Housing[bottom])

   # a boolean variable to check if the town is a university town
   f=Housing.index.isin(univ.index)

   # subset a dataframe of university towns
   uni=Housing.loc[f==True].dropna()

   # subset a dataframe of non-university towns
   non_uni=Housing.loc[f==False].dropna()

   # collect the mean of ratios for univ towns
   mean_uni=uni['ratio'].mean()

   # collect the mean of ratios for non-univ towns
   mean_non_uni=non_uni['ratio'].mean()

   # test if the ratios are statistically similar or different
   x=stats.ttest_ind(uni['ratio'], non_uni['ratio'])

   p=x[1]
   different=False
   better=''

   if (p<0.01):
      # statistically different
      different=True

   if (mean_uni < mean_non_uni):
      better='university town'
   else:
      better='non-university town'

   tup=(different,p,better)
   return tup

   # '''First creates new data showing the decline or growth of housing prices
    #between the recession start and the recession bottom. Then runs a ttest
    #comparing the university town values to the non-university towns values, 
    #return whether the alternative hypothesis (that the two groups are the same)
    #is true or not as well as the p-value of the confidence. 
    
    #Return the tuple (different, p, better) where different=True if the t-test is
    #True at a p<0.01 (we reject the null hypothesis), or different=False if 
    #otherwise (we cannot reject the null hypothesis). The variable p should
    #be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    #value for better should be either "university town" or "non-university town"
    #depending on which has a lower mean price ratio (which is equivilent to a
    #reduced market loss).'''
    
run_ttest()

(True, 0.005164866327919532, 'university town')